In [1]:
import os
import pandas as pd
os.system("bamplot -h")

usage: bamplot [options]

optional arguments:
  -h, --help            show this help message and exit
  -b [BAM [BAM ...]], --bam [BAM [BAM ...]]
                        Enter a comma separated list of .bam files to be
                        processed.
  -i INPUT, --input INPUT
                        Enter .gff or genomic region e.g. chr1:+:1-1000.
  -g GENOME, --genome GENOME
                        specify a genome, HG18,HG19,MM8,MM9,MM10 are currently
                        supported
  -o OUTPUT, --output OUTPUT
                        Enter the output folder.
  --stretch-input STRETCH_INPUT
                        Stretch the input regions to a minimum length in bp,
                        e.g. 10000 (for 10kb)
  -c COLOR, --color COLOR
                        Enter a colon separated list of colors e.g.
                        255,0,0:255,125,0, default samples the rainbow
  -s SENSE, --sense SENSE
                        Map to '+','-' or 'both' strands. Default maps to
       

0

In [2]:
import IPython 
IPython.sys_info()

"{'commit_hash': '24103c4',\n 'commit_source': 'installation',\n 'default_encoding': 'utf-8',\n 'ipython_path': '/opt/miniconda3/envs/my_env/lib/python3.10/site-packages/IPython',\n 'ipython_version': '8.2.0',\n 'os_name': 'posix',\n 'platform': 'Linux-4.14.158-101.185.amzn1.x86_64-x86_64-with-glibc2.17',\n 'sys_executable': '/opt/miniconda3/envs/my_env/bin/python',\n 'sys_platform': 'linux',\n 'sys_version': '3.10.4 | packaged by conda-forge | (main, Mar 24 2022, '\n                '17:39:04) [GCC 10.3.0]'}"

In [3]:
def bamplot_cmd_internal(gene,region):

    data_table = pd.read_table("/data/projects/IRF4_mm_trn_data/myeloma_public_data/tables/p300_interal_perturb_data_table.txt")
    qc_table = pd.read_csv("/data/projects/IRF4_mm_trn_data/myeloma_public_data/tables/MM_p300_internal_QC_report.csv")

    order = data_table['UNIQUE_ID'].astype(str).str[:2]
    order = order.astype(int)
    data_table['order'] = order
    data_table = data_table.sort_values(by='order')

    order = qc_table['Sample'].astype(str).str[:2]
    order = order.astype(int)
    qc_table['order'] = order
    qc_table = qc_table.sort_values(by='order')

    selected_columns = qc_table[['Sample', 'order','Scale_factor']]

    data_table = data_table.merge(selected_columns, on='order', how='inner')

    data_table['time'] = [0,0,1,1,3,3,6,6,24,24,0,0,1,1,3,3,6,6,24,24,0,0,1,1,3,3,6,6,24,24,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]

    data_table['bam'] = data_table['FILE_PATH'] +data_table['UNIQUE_ID']+".bam"

    tf_bams = data_table['bam'][30:34].str.cat(sep=',')

    n0 = ["H3K18Ac_0","H3K18Ac_0","H3K27Ac_0","H3K27Ac_0","H3K27me3_0","H3K27me3_0"]

    #region = "chr17:+:37847100-38060047"
    #gene =""

    cmd = "mkdir /home/ec2-user/IRF4_mm_trn/mm_public_chip_scripts/figures/" + gene
    os.system(cmd)
    o = "/home/ec2-user/IRF4_mm_trn/mm_public_chip_scripts/figures/"+ gene+"/"

    g = "hg19" 
    e = "2000" 
    c0 = "220,220,220:240,240,240"
    ct = "90,90,90:65,65,65"
    c = c0 +":"+ ct
    
    np300 = "IRF4,IRF4,p300,p300"
    
    cp300 = "244,64,247:230,70,243:51,232,232:45,208,208"
    y = "UNIFORM"
    p = "MERGE"

    i = region
    title = gene+"_tfs_bamplot"

    
    cmd = 'bamplot -g %s -e %s -c %s -o %s -y %s -b %s -i %s -n %s -t %s -r --save-temp -p %s \n\n' % (g,e,cp300,o,y,tf_bams,i,np300,title,p)
    print(cmd)
    os.system(cmd)
    
    filtered_df = data_table[(data_table.time == 0)]
    t0_bams = filtered_df['bam'][0:6].str.cat(sep=',')
    t0_sf = filtered_df['Scale_factor'].astype('str').str.cat(sep=',')

    n_0 = pd.Series(n0)
    n_0 = n_0.str.cat(sep=',')
    
    title = gene+"_bamplot_internal"
    cmd = 'bamplot -g %s -e %s -c %s -o %s -y %s -b %s -i %s -n %s -t %s -r --save-temp -p %s \n\n' % (g,e,c,o,y,t0_bams,i,n_0,title,p)
    print(cmd)
    os.system(cmd)
    

In [4]:
def bamplot_cmd_public(gene,region):
    data_table = pd.read_table("/data/projects/IRF4_mm_trn_data/myeloma_public_data/tables/mm_h3k27ac_table.txt")

    data_table['bam'] = data_table['FILE_PATH'] +data_table['UNIQUE_ID']+".bam"

    bams = data_table['bam'].str.cat(sep=',')
    
    o = "/home/ec2-user/IRF4_mm_trn/mm_public_chip_scripts/figures/"+ gene+"/"

    g = "hg19" 
    e = "2000" 

    names = "cell_line,cell_line,cell_line,cell_line,cell_line,cell_line,cell_line,cell_line,cell_line,cell_line,patient,patient,patient,patient,cell_line,patient,patient,patient,patient,patient,patient,cell_line,cell_line,cell_line,cell_line,cell_line,cell_line,patient,patient,patient,patient,patient,patient,patient,patient,patient,patient"

    y = "UNIFORM"
    p = "MERGE"

    i = region
    title = gene+"_public_H3K27ac_bamplot"

    cmd = 'bamplot -g %s -e %s -o %s -y %s -b %s -i %s -n %s -t %s -r --save-temp -p %s \n\n' % (g,e,o,y,bams,i,names,title,p)

    print(cmd)
    os.system(cmd)


In [5]:
region = "chr6:+:261715-458764"
gene ="IRF4"
bamplot_cmd_internal(gene,region)
bamplot_cmd_public(gene,region)

bamplot -g hg19 -e 2000 -c 244,64,247:230,70,243:51,232,232:45,208,208 -o /home/ec2-user/IRF4_mm_trn/mm_public_chip_scripts/figures/IRF4/ -y UNIFORM -b /data/projects/p300_internal_mm/31_0EX7_0229Kronos_MM1S-No-Treatment-1_IRF4_hs_i45.bam,/data/projects/p300_internal_mm/32_0EX8_0229Kronos_MM1S-No-Treatment-2_IRF4_hs_i48.bam,/data/projects/p300_internal_mm/33_0EX9_0229Kronos_MM1S-No-Treatment-1_p300_hs_i49.bam,/data/projects/p300_internal_mm/34_0EXA_0229Kronos_MM1S-No-Treatment-2_p300_hs_i50.bam -i chr6:+:261715-458764 -n IRF4,IRF4,p300,p300 -t IRF4_tfs_bamplot -r --save-temp -p MERGE 




mkdir: cannot create directory ‘/home/ec2-user/IRF4_mm_trn/mm_public_chip_scripts/figures/IRF4’: File exists


null device 
          1 
	Loading genome data from cache.
bamplot -g hg19 -e 2000 -c 220,220,220:240,240,240:90,90,90:65,65,65 -o /home/ec2-user/IRF4_mm_trn/mm_public_chip_scripts/figures/IRF4/ -y UNIFORM -b /data/projects/p300_internal_mm/01_0EWN_0229Kronos_MM1S-No-Treatment-1_H3K18Ac_hs-dm_i02.bam,/data/projects/p300_internal_mm/02_0EWO_0229Kronos_MM1S-No-Treatment-2_H3K18Ac_hs-dm_i04.bam,/data/projects/p300_internal_mm/11_0EWX_0229Kronos_MM1S-No-Treatment-1_H3K27Ac_hs-dm_i20.bam,/data/projects/p300_internal_mm/12_0EWY_0229Kronos_MM1S-No-Treatment-2_H3K27Ac_hs-dm_i21.bam,/data/projects/p300_internal_mm/21_0F27_0229Kronos_MM1S-No-Treatment-1_H3K27me3_hs-dm_i02.bam,/data/projects/p300_internal_mm/22_0F28_0229Kronos_MM1S-No-Treatment-2_H3K27me3_hs-dm_i04.bam -i chr6:+:261715-458764 -n H3K18Ac_0,H3K18Ac_0,H3K27Ac_0,H3K27Ac_0,H3K27me3_0,H3K27me3_0 -t IRF4_bamplot_internal -r --save-temp -p MERGE 


null device 
          1 
	Loading genome data from cache.
bamplot -g hg19 -e 2000 -o /hom